In [11]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [48]:
# load the data set using pandas
data=pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\oil_prices_yahoo1.csv')

In [50]:
# covertnig the date column to datetime using pandas
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# creating a data frame using pandas
df=pd.DataFrame(data)

# renaming the column to price
df.rename(columns = {'Close':'Price'}, inplace = True)

In [14]:
print(df.head())
print(df.tail())

        Date      Price
0 2000-08-23  32.049999
1 2000-08-24  31.629999
2 2000-08-25  32.049999
3 2000-08-28  32.869999
4 2000-08-29  32.720001
           Date      Price
5746 2023-07-17  74.150002
5747 2023-07-18  75.750000
5748 2023-07-19  75.349998
5749 2023-07-20  75.629997
5750 2023-07-21  77.070000


In [15]:
import pandas as pd
from scipy import stats

# Convert the date column to a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Group the data by month
grouped = df.groupby(pd.Grouper(key='Date', freq='M'))

# Define a function to detect outliers within each group
def detect_outliers(group):
    z_scores = stats.zscore(group['Price'])
    outliers = group[(z_scores < -3) | (z_scores > 3)]
    return outliers

# Apply the function to each group
outliers = grouped.apply(detect_outliers)

# View the outliers
print(outliers)


                      Date      Price
Date                                 
2004-06-30 938  2004-06-01  42.330002
2014-11-30 3578 2014-11-28  66.150002
2019-05-31 4708 2019-05-31  53.500000
2020-04-30 4931 2020-04-20 -37.630001
2021-01-31 5110 2021-01-04  47.619999


In [16]:
import pandas as pd
from tsmoothie.smoother import *
from tsmoothie.utils_func import create_windows

# create a smoother instance
smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)

# operate the smoothing on the 'Price' column
smoother.smooth(df['Price'])

# generate intervals
low, up = smoother.get_intervals('prediction_interval')

# find the indexes of the outliers
outliers = np.argwhere((df['Price'].values < low) | (df['Price'].values > up)).ravel()

# print the outliers
print(outliers)


[   0 1910    0 1911    0 1912    0 1913    0 1914    0 1915    0 1916
    0 1917    0 1918    0 1919    0 1920    0 1922    0 1923    0 1924
    0 1925    0 1926    0 1927    0 1928    0 1929    0 1930    0 1931
    0 1932    0 1933    0 1934    0 1935    0 1936    0 1937    0 1938
    0 1939    0 1940    0 1941    0 1942    0 1943    0 1944    0 1945
    0 1946    0 1947    0 1948    0 1949    0 1950    0 1951    0 1952
    0 1953    0 1954    0 1955    0 1956    0 1957    0 1958    0 1959
    0 1960    0 1961    0 1962    0 1963    0 1964    0 1965    0 1966
    0 1967    0 1968    0 1969    0 1970    0 1971    0 1972    0 1973
    0 1974    0 1975    0 1976    0 1977    0 1978    0 1979    0 1980
    0 1981    0 1982    0 1983    0 1984    0 1985    0 1986    0 1987
    0 1988    0 1989    0 1990    0 1991    0 1992    0 1993    0 1994
    0 1995    0 1996    0 1997    0 1998    0 1999    0 2000    0 2001
    0 2002    0 2003    0 2004    0 2005    0 2006    0 2007    0 2008
    0 

In [17]:
import pandas as pd
import numpy as np


# calculate the upper and lower limits for capping
upper_limit = df['Price'].mean() + 3 * df['Price'].std()
lower_limit = df['Price'].mean() - 3 * df['Price'].std()

# find the outliers
outliers = df[(df['Price'] > upper_limit) | (df['Price'] < lower_limit)]

# print the outliers
print(outliers)

# apply capping to the 'Price' column
df['Price'] = np.where(df['Price'] > upper_limit, upper_limit, df['Price'])
df['Price'] = np.where(df['Price'] < lower_limit, lower_limit, df['Price'])



           Date       Price
1963 2008-07-01  140.970001
1964 2008-07-02  143.570007
1965 2008-07-03  145.289993
1966 2008-07-07  141.369995
1969 2008-07-10  141.649994
1970 2008-07-11  145.080002
1971 2008-07-14  145.179993
4931 2020-04-20  -37.630001


In [21]:
# Using IQR Method

import pandas as pd

# calculate the IQR
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

# define the upper and lower bounds for outliers
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR

# find the outliers
outliers = df[(df['Price'] > upper_bound) | (df['Price'] < lower_bound)]
print(outliers)

# save the outliers in a new DataFrame
outliers_df = pd.DataFrame(outliers)

# remove the outliers from the original DataFrame
df_IQR = df[(df['Price'] <= upper_bound) & (df['Price'] >= lower_bound)]


           Date       Price
1964 2008-07-02  143.570007
1965 2008-07-03  145.289993
1970 2008-07-11  145.080002
1971 2008-07-14  145.179993
4931 2020-04-20  -37.630001


In [34]:
import pandas as pd
from scipy import stats

# create a copy of the original DataFrame
new_df = df.copy()

# calculate the Z-scores
z_scores = stats.zscore(new_df['Price'])

# define the threshold for outliers
threshold = 3

# find the indexes of the outliers
outlier_indexes = new_df[(z_scores > threshold) | (z_scores < -threshold)].index
print(outlier_indexes)

# recode the outliers to the mean of the data in the new DataFrame
new_df.loc[outlier_indexes, 'Price'] = new_df['Price'].median()


Int64Index([1963, 1964, 1965, 1966, 1969, 1970, 1971, 4931], dtype='int64')


In [38]:
import pandas as pd
from scipy import stats

# calculate the Z-scores
z_scores = stats.zscore(new_df['Price'])

# define the threshold for outliers
threshold = 3

# find the outliers
outliers = new_df[(z_scores > threshold) | (z_scores < -threshold)]

# save the outliers in a new DataFrame
outliers_df = pd.DataFrame(outliers)

print(outliers_df)


           Date       Price
1961 2008-06-27  140.210007
1962 2008-06-30  140.000000


In [42]:
import pandas as pd
from sklearn.ensemble import IsolationForest

# create a copy of the original DataFrame
new_dfi = df.copy()

# create an Isolation Forest model
iso_forest = IsolationForest(contamination=0.1)

# fit the model to the 'Price' column
iso_forest.fit(new_dfi[['Price']])

# predict outliers
outlier_predictions = iso_forest.predict(new_dfi[['Price']])

# find the indexes of the outliers
outlier_indexes = new_dfi[outlier_predictions == -1].index

# recode the outliers to the mean of the data in the new DataFrame
new_dfi.loc[outlier_indexes, 'Price'] = new_dfi['Price'].median()



In [44]:
print(outlier_indexes)
print(new_dfi)

Int64Index([  13,   15,   20,   41,   55,   56,   57,   66,   67,   82,
            ...
            5478, 5479, 5481, 5482, 5483, 5484, 5485, 5486, 5490, 5491],
           dtype='int64', length=575)
           Date      Price
0    2000-08-23  32.049999
1    2000-08-24  31.629999
2    2000-08-25  32.049999
3    2000-08-28  32.869999
4    2000-08-29  32.720001
...         ...        ...
5746 2023-07-17  74.150002
5747 2023-07-18  75.750000
5748 2023-07-19  75.349998
5749 2023-07-20  75.629997
5750 2023-07-21  77.070000

[5751 rows x 2 columns]


In [51]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# create a copy of the original DataFrame
new_dfl = df.copy()

# create a LOF model
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

# fit the model to the 'Price' column
lof.fit(new_dfl[['Price']])

# predict outliers
outlier_predictions = lof.fit_predict(new_dfl[['Price']])

# find the indexes of the outliers
outlier_indexes = new_dfl[outlier_predictions == -1].index

# recode the outliers to the mean of the data in the new DataFrame
new_dfl.loc[outlier_indexes, 'Price'] = new_dfl['Price'].median()


In [52]:
print(outlier_indexes)

Int64Index([   5,    7,   21,   43,   50,   51,   53,  101,  109,  127,
            ...
            5578, 5586, 5600, 5629, 5648, 5681, 5704, 5716, 5718, 5723],
           dtype='int64', length=575)
